In [18]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import yaml

In [19]:
df_subdist_shape = gpd.read_file('/home/inductive-anks/yield-prediction/Yield-Prediction/data/All-India-Subdistrict-Shape-File/SUBDISTRICT_BOUNDARY.shp')

In [20]:
df_village_list = pd.read_csv('/home/inductive-anks/yield-prediction/Yield-Prediction/data/Tehsil Data/Tehsil_Data_With_Single_Village.csv')

In [21]:
df_SOI_shape = gpd.read_file('/home/inductive-anks/yield-prediction/Yield-Prediction/data/All-Inida-SOI/All-India-Villages-SOI.shp')

In [22]:
df_SOI_shape.shape

(599742, 5)

In [23]:
df_SOI_shape.head()

,NAME,dtnmcoc,state,tehnmcoc,geometry
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101..."
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775..."
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418..."
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894..."
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541..."


In [27]:
df_SOI_shape.rename(columns={'NAME': 'SOI_Village_Name', 'dtnmcoc': 'SOI_District_Name', 'tehnmcoc':'SOI_Tehsil_Name'}, inplace=True)

In [28]:
df_SOI_shape.head()

,SOI_Village_Name,SOI_District_Name,state,SOI_Tehsil_Name,geometry
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101..."
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775..."
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418..."
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894..."
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541..."


In [24]:
df_village_list.head()

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
0,2008-2011,andhra pradesh,ranga reddy,IV,Puduru,40,pedhaumethala
1,2008-2011,andhra pradesh,medak,V,Sadasivpet,49,nandi kandi
2,2008-2011,andhra pradesh,medak,V,Zaiheerabad,50,huggelly
3,2008-2011,andhra pradesh,srikakulam,I,Etheria,1,ponnada
4,2008-2011,andhra pradesh,srikakulam,I,Tekkali,2,tirlangi


In [25]:
df_subdist_shape.head()

,District,STATE,TEHSIL,Shape_Leng,Shape_Area,geometry
0,BIL>SPUR,HIMACHAL PRADESH,BHARARI,50253.228763,9.000306e+07,"POLYGON ((3690806.89 4831388.917, 3690871.421 ..."
1,BIL>SPUR,HIMACHAL PRADESH,BIL>SPUR,93446.685818,1.703525e+08,"POLYGON ((3699861.726 4812292.061, 3699965.144..."
2,BIL>SPUR,HIMACHAL PRADESH,GHURMW|N,105834.665205,1.906542e+08,"POLYGON ((3692609.535 4827764.245, 3692614.853..."
3,BIL>SPUR,HIMACHAL PRADESH,JHAND@TA,112831.475031,3.205376e+08,"POLYGON ((3667564.424 4819909.775, 3667603.174..."
4,BIL>SPUR,HIMACHAL PRADESH,NAMHOL,61752.616994,1.034979e+08,"POLYGON ((3701617.644 4803943.439, 3701666.832..."


## Converting to Lower Case

In [29]:
df_subdist_shape = df_subdist_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_23604/228283043.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_subdist_shape = df_subdist_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [30]:
df_village_list = df_village_list.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_23604/990218482.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_village_list = df_village_list.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [31]:
df_SOI_shape = df_SOI_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_23604/3172920881.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_SOI_shape = df_SOI_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)


## Using Jaro Similarity to Filter SOI Tehsils using Village List

In [33]:
df_village_list.head()

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
0,2008-2011,andhra pradesh,ranga reddy,iv,puduru,40,pedhaumethala
1,2008-2011,andhra pradesh,medak,v,sadasivpet,49,nandi kandi
2,2008-2011,andhra pradesh,medak,v,zaiheerabad,50,huggelly
3,2008-2011,andhra pradesh,srikakulam,i,etheria,1,ponnada
4,2008-2011,andhra pradesh,srikakulam,i,tekkali,2,tirlangi


In [39]:
df_SOI_shape.sample(3)

,SOI_Village_Name,SOI_District_Name,state,SOI_Tehsil_Name,geometry
390484,padhanpada,kandhamal,orissa,tikabali,"POLYGON ((84.28775 20.28149, 84.28834 20.2815,..."
35057,dosaledu,anantapur,andhra pradesh,putlur,"POLYGON ((77.87422 14.78846, 77.87814 14.78956..."
351608,arniya mali,chittaurgarh,rajasthan,nimbahera,"POLYGON ((74.69381 24.643, 74.69409 24.64295, ..."


In [46]:
import pandas as pd
from nltk.metrics.distance import jaro_similarity

# Initialize an empty list to store the results
results = []

# Iterate over each row in df_village_list
for _, village_row in df_village_list.iterrows():
    state = village_row['State Name']
    tehsil = village_row['Tehsil Name']
    village_name = village_row['Village Name']
    
    # Skip if village_name is None or empty
    if village_name is None or not len(village_name.strip()):
        continue
    
    # Filter df_SOI_shape by state and tehsil
    matching_SOI = df_SOI_shape[(df_SOI_shape['state'] == state) & 
                                (df_SOI_shape['SOI_Tehsil_Name'] == tehsil)]
    
    # Iterate over the filtered df_SOI_shape
    for _, soi_row in matching_SOI.iterrows():
        soi_village_name = soi_row['SOI_Village_Name']
        
        # Skip if soi_village_name is None or empty
        if soi_village_name is None or not len(soi_village_name.strip()):
            continue
        
        # Calculate Jaro similarity
        similarity = jaro_similarity(village_name, soi_village_name)
        
        # Check if the similarity is above the threshold
        if similarity >= 0.8:
            # Combine the information from both dataframes
            combined_row = {
                **village_row.to_dict(),
                **soi_row.to_dict(),
                'Jaro Similarity': similarity
            }
            results.append(combined_row)

# Convert the results into a DataFrame
result_df = pd.DataFrame(results)


In [47]:
column_order = [
    'Year', 'State Name', 'state', 'District Name', 'SOI_District_Name', 'Zone No', 'Tehsil Name', 'SOI_Tehsil_Name', 'Tehsil Code', 'Village Name',
    'SOI_Village_Name', 'Jaro Similarity', 'geometry'
]

# Reorder the dataframe
result_df = result_df[column_order]

In [48]:
result_df.shape

(1793, 13)

In [55]:
result_df.head(2)

,Year,State Name,state,District Name,SOI_District_Name,Zone No,Tehsil Name,SOI_Tehsil_Name,Tehsil Code,Village Name,SOI_Village_Name,Jaro Similarity,geometry
0,2008-2011,andhra pradesh,andhra pradesh,medak,medak,v,sadasivpet,sadasivpet,49,nandi kandi,nandikandi,0.969697,"POLYGON ((77.98548 17.62885, 77.9872 17.62891,..."
1,2008-2011,andhra pradesh,andhra pradesh,srikakulam,srikakulam,i,tekkali,tekkali,2,tirlangi,thirlangi,0.962963,"POLYGON ((84.20608 18.61308, 84.20653 18.61301..."


In [54]:
df_subdist_shape.head(3)

,District,STATE,TEHSIL,Shape_Leng,Shape_Area,geometry
0,bil>spur,himachal pradesh,bharari,50253.228763,9.000306e+07,"POLYGON ((3690806.89 4831388.917, 3690871.421 ..."
1,bil>spur,himachal pradesh,bil>spur,93446.685818,1.703525e+08,"POLYGON ((3699861.726 4812292.061, 3699965.144..."
2,bil>spur,himachal pradesh,ghurmw|n,105834.665205,1.906542e+08,"POLYGON ((3692609.535 4827764.245, 3692614.853..."


In [68]:
import geopandas as gpd

result_gdf = gpd.GeoDataFrame(result_df, geometry='geometry')

subdist_gdf = gpd.GeoDataFrame(df_subdist_shape, geometry='geometry')

if result_gdf.crs is None:
    result_gdf = result_gdf.set_crs('EPSG:4326') 

if subdist_gdf.crs is None:
    subdist_gdf = subdist_gdf.set_crs('EPSG:4326') 

result_gdf = result_gdf.to_crs('EPSG:4326')
subdist_gdf = subdist_gdf.to_crs('EPSG:4326')

merged_gdf = gpd.sjoin(result_gdf, subdist_gdf, how="inner", predicate='within')


In [69]:
merged_gdf.shape

(1145, 19)